## 7.4 枚举

一个`enum`只不过是一个伪装的整数。枚举和整数有一样效率。

请注意，枚举（值名称）将与具有相同名称的任何变量或函数冲突。
因此，头文件中的枚举应具有长且唯一的枚举器名称，或放入命名空间中。

## 7.5 布尔类型

### 布尔运算的顺序

布尔运算符`&&`和`||`的操作数，按以下方式进行计算。
- 如果&&的第一个操作数为`false`，那么第二个操作数根本就不计算，因为无论第二个操作数的值如何，结果都是`false`。
- 同样，如果`||`的第一个操作数是`true`，那么第二个操作数就不会被评估，因为结果无论如何都是`true`。

将最可能值为`true`的操作数放在`&&`表达式的最后一个，或者放在`||`表达式的第一个，这可能是对性能有利的。
例如，假设`a`为`true`的时间为50%，`b`为`true`时间为10%。
- 当`a`为`true`时，表达式`a && b`还需要评估`b`，这占50％的情况。
- 等价表达式`b && a`只需在`b`为`true`时评估，这只占10％的情形。
如果`a`和`b`分别需要相同的时间进行评估，并且可能通过分支预测机制进行预测，则（第二种方法）速度会更快。
有关分支预测的解释，请参阅第44页。

**如果一个操作数比另一个操作数更具可预测性，那么把更可预测的操作数放前面。**

**如果一个操作数比另一个操作数计算得更快，那么首先把最快计算得出的操作数放在前面。**

但是，在交换布尔操作数的顺序时必须小心。
如果操作数的评估有副作用，或者需要第一个操作数确定第二个操作数是否有效，则不能交换操作数。 例如：

```cpp
// Example 7.7
unsigned int i;
const int ARRAYSIZE = 100;
float list[ARRAYSIZE];
if (i < ARRAYSIZE && list[i] > 1.0) { ...
```

这里，你不能交换操作数的顺序，因为当变量`i`不小于ARRAYSIZE时，表达式列表`[i]`是无效的。 另一个例子：

```cpp
// Example 7.8
if (handle != INVALID_HANDLE_VALUE && WriteFile(handle, ...)) { ...
```

这里，你不能交换布尔操作数的顺序，因为如果句柄无效，你不应该调用`WriteFile`。


### 布尔变量是超定的

布尔变量存储为8位整数，值为0代表假，1代表真。

布尔变量是超定的（过度确认），因为
- 所有具有布尔变量作为输入的运算符都检查输入是否具有除0或1以外的其他值，
- 但具有布尔值作为输出的运算符不能产生除0或1之外的其他值。

这使得使用布尔作为输入变量，操作的效率有些不必要的要低。举个例子：

```cpp
// Example 7.9a
bool a, b, c, d;
c = a && b;
d = a || b;
```


This is typically implemented by the compiler in the following way:
```cpp
bool a, b, c, d;
if (a != 0) {
    if (b != 0) {
        c = 1;
    }
    else {
        goto CFALSE;
     }
}
else {
    CFALSE:
    c = 0;
}
if (a == 0) {
    if (b == 0) {
    d = 0;
    }
    else {
        goto DTRUE;
    }
}
else {
    DTRUE:
    d = 1;
}
```

这段代码显然离最优还差得很远。
如果发现预测失误，分支可能需要很长时间（请参阅第44页）。
如果可以肯定地知道操作数没有0和1以外的其它值，布尔操作可以更有效率。
编译器之所以没有做出这种假设，这是因为，如果变量未初始化或来自未知来源，则变量可能具有其它值。
如果`a`和`b`已经初始化为有效值，或者它们来自产生布尔输出的运算符，则上面的代码可以被优化。
优化的代码如下所示：

```cpp
// Example 7.9b
char a = 0, b = 0, c, d;
c = a & b;
d = a | b;
```

在这里，我使用了`char`（或`int`）而不是`bool`，以便使用位运算符（`&`和`|`）而不是布尔运算符（`&&`和`||`）。
按位运算符是只需要一个时钟周期的一组单指令。
即使`a`和`b`的值不等于0或1，`OR`运算符（`|`）也能工作。
如果操作数的值不等于0和1，`AND`运算符（`&`）和异或运算符（`^`）可能会有除0和1之外的其它结果。

请注意，这里有一些坑。 `NOT`不能使用`~`。
替代的方法是，你可以在已知为0或1的变量上通过与1进行异或运算来取反：

```cpp
// Example 7.10a
bool a, b;
b = !a;
```
可以被优化为：
```cpp
// Example 7.10b
char a = 0, b;
b = a ^ 1;
```

当a为假，如果b是一个不应被评估的表达式，则不能用`a & b`替换`a && b`。
同样，当a为真，如果b是一个表达式，你不能把`a || b`替换为`a | b`，则该表达式也不应被评估。

操作数是变量的情况，比起操作数是比较等表达式的情况，使用按位运算符的技巧等更有利。例如：

```cpp
// Example 7.11
bool a; float x, y, z;
a = x > y && z != 0;
```

这段代码在大多数情况下是最优的。除非预计`&&`表达式会产生很多分支预测错误，否则请勿把`&&`改成`&`。

### 布尔向量操作

一个整数可以用作布尔向量。
例如，如果`a`和`b`是32位整数，则表达式`y = a & b;` 将在一个时钟周期内完成32个与操作。
运算符 `&`, `|`, `^`, `~` 对布尔向量运算很有用。